In [ ]:
import torch
from torchvision import datasets, transforms
import helper

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor,
                                transforms.Normalize([0.5], [0.5])])

In [ ]:
import pandas as pd

In [ ]:
# reads in the climate data
data = pd.read_csv('hack-data - Sheet1.csv')

In [93]:

from torch.utils.data import Dataset, DataLoader

# creates a dataset
class MyDataset(Dataset):
    def __init__(self, root, n_inp, transform=None):
        self.df = pd.read_csv(root)
        self.data = self.df.to_numpy()
        self.x , self.y = (torch.from_numpy(self.data[:,:n_inp]),
                           torch.from_numpy(self.data[:,n_inp:]))
        self.x = self.x.type(torch.float32)
        self.y = self.y.type(torch.float32)
        self.transform=transform
    def __getitem__(self, idx):
        x = self.x[idx, :]
        y = self.y[idx,:]

        return x,y
    def __len__(self):
        return len(self.data)

# creates dataset object
myData = MyDataset("hack-data - Sheet1.csv", 9, transform=transform)

# creates data loader
data_loader = DataLoader(myData, batch_size=4, shuffle =True)


In [94]:
from torch import nn, optim
import torch.nn.functional as F

n_in = 9 # climate 
n_out = 3

# TODO: Define your network architecture here
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()

        # define the actual model
        self.fc1 = nn.Linear(n_in, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, n_out)
        
    def forward(self, x):
        # run an input data point through the model
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x

In [95]:
# create the model, define the loss and the optimizer
model = Classifier()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [97]:
# create the training loop for the model to learn
epochs = 500

for e in range(epochs):
    running_loss = 0
    for data, labels in data_loader:
        log_ps = model(data.float())
        loss = criterion(log_ps.float(), labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(data_loader)}")

Training loss: 23386.78025390625
Training loss: 20566.840126953124
Training loss: 19554.133466796877
Training loss: 18731.6385546875
Training loss: 19330.568657226562
Training loss: 17680.21743652344
Training loss: 17447.264853515626
Training loss: 17430.716884765625
Training loss: 17930.595161132813
Training loss: 17629.9675390625
Training loss: 15732.4459765625
Training loss: 17437.715068359375
Training loss: 15571.157705078125
Training loss: 18404.282983398436
Training loss: 15953.117143554688
Training loss: 16169.9787109375
Training loss: 16851.679321289062
Training loss: 17004.385859375
Training loss: 15671.531586914063
Training loss: 17215.71923828125
Training loss: 15493.48505859375
Training loss: 16755.094501953125
Training loss: 14687.8016015625
Training loss: 16043.124580078125
Training loss: 16091.5494140625
Training loss: 14066.00439453125
Training loss: 14858.73177734375
Training loss: 14634.729013671875
Training loss: 14388.631015625
Training loss: 15159.214672851562
Trai

In [109]:
import numpy as np

# example input:
# takes in an array of format [Temperature, Rainfall, Humidity (%),	Wind Speed,	Crop Yield,	
#                              Acreage,	Nitrogen Soil Presence,	Potassium Soil Presence,	
#                              Phosphorus Soil Presence]

# create the example input
sample_data = np.array([67,	12,	0.7,	12,	32000,	100,	0.35,	0.75,	0.32])
sample_data = sample_data.reshape((9,1)).T

# preprocessing
sample_data = torch.from_numpy(sample_data).type(torch.float32)

# get the output from the model
out = model(sample_data)

In [110]:
# print the output
print(out)

tensor([[138.4817, 209.5386, 208.7975]], grad_fn=<AddmmBackward>)


In [111]:
# save the model parameters
torch.save(model.state_dict(), 'model_params')